In [2]:
import pandas as pd
import numpy as np
import os

import acquire as a 
import prepare as p

from env import host, username, password
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'env'

In [3]:
df = p.acquire()

NameError: name 'p' is not defined